# BLAST
---

1. Download Data:
    1. [Human Genome Resources at NCBI](https://www.ncbi.nlm.nih.gov/projects/genome/guide/human/index.shtml) or [Genome assembly GRCh38.p14](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_000001405.26/)
    2. [大腸菌ゲノムデータ](https://www.ncbi.nlm.nih.gov/data-hub/taxonomy/562/)

2. Create two BLAST databases(two ways)
   1. CommandLine
       `makeblastdb -in <reference.fa> -dbtype prot -parse_seqids -out <database_name> -title "Database title"`
   2. [Wrapper by biopython](https://biopython.org/docs/1.75/api/Bio.Blast.Applications.html#Bio.Blast.Applications.NcbimakeblastdbCommandline)

3. Create forward and reverse BLAST searches (XML)
    I will restrict the number of matches to be returned to 1, by using the -max_target_seqs argument and -max_hsps argument. This will only report the best database hit to each query. We will not need to build a separate "best hit table". And I will take the 'best HSP'
    ```
    import subprocess
    cmd = "blastp -query GRCh38_latest_protein.faa -out human.blast -db ecli_protein -outfmt 6 -num_threads 8 -max_hsps 1 -max_target_seqs 500  -mt_mode 1"
    cmd += " -subject_besthits"
    subprocess.run(cmd, shell=True)
    ```
    逆も然り
    Or use xml to process, generates format 5
   
----

Thanks to https://www.youtube.com/watch?v=bN5p1mKLxxo


## Extract hsps(Bit score > 60)

In [1]:
from Bio import SearchIO
import pandas as pd
bitscore_sixty = []
for blast_record in SearchIO.parse('data/human.tab', 'blast-tab'):
    if blast_record.hsps[0].bitscore > 60.0:
        bitscore_sixty.append([blast_record.id, blast_record.hsps[0].hit_id, blast_record.hsps[0].bitscore])
print(bitscore_sixty[:10])

[['NP_000007.1', 'NP_308069.1', 154.0], ['NP_000008.1', 'NP_310429.2', 176.0], ['NP_000009.1', 'NP_308069.1', 130.0], ['NP_000010.1', 'NP_311728.2', 328.0], ['NP_000013.2', 'NP_310358.1', 159.0], ['NP_000022.3', 'NP_308450.2', 230.0], ['NP_000023.2', 'NP_312522.1', 219.0], ['NP_000024.2', 'NP_310128.2', 101.0], ['NP_000028.3', 'NP_308394.1', 70.9], ['NP_000036.2', 'NP_311839.1', 73.2]]


## Bidirectional best hits
>   Finding best hits involved sorting the results for a query-genome-to-subject-genome comparison from highest to lowest score. The first hit for each query protein within the sorted results would therefore be the best hit. If the next hit had the very same score there would be more than one best hit (the method can therefore produce co-orthologs). We performed the very same procedure for the results ran in the opposite direction. That is, for the results where the subject genome was used as a query, and the query genome was used as a subject. Finally, to find orthologs as reciprocal best hits, for each best hit found by a query protein in the first direction, we checked if it found this query gene as a best hit in the opposite direction.

In [2]:
fwd_results = pd.read_csv('data/human.tab', sep="\t", header=None)
rev_results = pd.read_csv('data/ecoli.tab', sep="\t", header=None)
headers = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
fwd_results.columns = headers
rev_results.columns = headers
# Merge forward and reverse results
bbh = pd.merge(fwd_results, rev_results[['qseqid', 'sseqid']],
                left_on='sseqid', right_on='qseqid',
                how='inner')
bbh.head(10)

,qseqid_x,sseqid_x,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qseqid_y,sseqid_y
0,NP_000005.3,NP_311413.1,28.889,135,93,3,130,263,389,521,8.380000e-07,50.8,NP_311413.1,NP_001334352.2
1,NP_000005.3,NP_311413.1,23.810,84,61,2,969,1052,1184,1264,1.100000e+00,30.8,NP_311413.1,NP_001334352.2
2,NP_000006.2,NP_310912.1,31.944,72,40,3,43,107,298,367,9.100000e-01,28.1,NP_310912.1,NP_003045.2
3,NP_000007.1,NP_308069.1,28.021,389,264,6,38,420,2,380,5.980000e-44,154.0,NP_308069.1,NP_000008.1
4,NP_000008.1,NP_310429.2,30.548,383,257,7,34,412,5,382,3.530000e-52,176.0,NP_310429.2,NP_000008.1
5,NP_000009.1,NP_308069.1,29.659,381,242,10,94,467,5,366,1.170000e-33,130.0,NP_308069.1,NP_000008.1
6,NP_000010.1,NP_311728.2,43.622,392,216,3,39,426,1,391,1.670000e-110,328.0,NP_311728.2,NP_005882.2
7,NP_000011.2,NP_310407.1,37.209,43,22,1,165,202,476,518,7.200000e-01,29.6,NP_310407.1,XP_005247819.1
8,NP_000012.1,NP_313059.2,31.429,70,37,2,198,267,12,70,1.100000e+00,26.6,NP_313059.2,XP_047287557.1
9,NP_000013.2,NP_310358.1,33.038,339,203,8,10,343,7,326,8.590000e-47,159.0,NP_310358.1,NP_000013.2


In [3]:
# Group duplicate BBH rows, taking the maximum value in each column.
bbh = bbh.groupby(['qseqid_x', 'sseqid_x'], as_index=False).max()
bbh.head(10)

,qseqid_x,sseqid_x,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qseqid_y,sseqid_y
0,NP_000005.3,NP_311413.1,28.889,135,93,3,969,1052,1184,1264,1.100000e+00,50.8,NP_311413.1,NP_001334352.2
1,NP_000006.2,NP_310912.1,31.944,72,40,3,43,107,298,367,9.100000e-01,28.1,NP_310912.1,NP_003045.2
2,NP_000007.1,NP_308069.1,28.021,389,264,6,38,420,2,380,5.980000e-44,154.0,NP_308069.1,NP_000008.1
3,NP_000008.1,NP_310429.2,30.548,383,257,7,34,412,5,382,3.530000e-52,176.0,NP_310429.2,NP_000008.1
4,NP_000009.1,NP_308069.1,29.659,381,242,10,94,467,5,366,1.170000e-33,130.0,NP_308069.1,NP_000008.1
5,NP_000010.1,NP_311728.2,43.622,392,216,3,39,426,1,391,1.670000e-110,328.0,NP_311728.2,NP_005882.2
6,NP_000011.2,NP_310407.1,37.209,43,22,1,165,202,476,518,7.200000e-01,29.6,NP_310407.1,XP_005247819.1
7,NP_000012.1,NP_313059.2,31.429,70,37,2,198,267,12,70,1.100000e+00,26.6,NP_313059.2,XP_047287557.1
8,NP_000013.2,NP_310358.1,33.038,339,203,8,10,343,7,326,8.590000e-47,159.0,NP_310358.1,NP_000013.2
9,NP_000014.1,NP_311396.1,41.935,31,14,1,15,45,298,324,7.900000e+00,25.8,NP_311396.1,NP_003866.1


In [4]:
# Discard rows that are not BBH
bbh = bbh.loc[bbh.qseqid_x == bbh.sseqid_y]
bbh.head(10)

,qseqid_x,sseqid_x,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qseqid_y,sseqid_y
3,NP_000008.1,NP_310429.2,30.548,383,257,7,34,412,5,382,3.530000e-52,176.0,NP_310429.2,NP_000008.1
8,NP_000013.2,NP_310358.1,33.038,339,203,8,10,343,7,326,8.590000e-47,159.0,NP_310358.1,NP_000013.2
24,NP_000033.2,NP_310944.1,33.333,54,30,1,184,237,26,73,4.200000e-02,32.0,NP_310944.1,NP_000033.2
60,NP_000074.3,NP_310325.2,27.273,121,77,4,456,574,294,405,2.820000e-05,44.7,NP_310325.2,NP_000074.3
74,NP_000088.3,NP_311334.1,47.195,303,150,4,151,453,7,299,1.460000e-90,274.0,NP_311334.1,NP_000088.3
83,NP_000099.2,NP_308147.1,43.612,454,249,6,40,493,5,451,6.930000e-115,345.0,NP_308147.1,NP_000099.2
89,NP_000107.1,NP_312540.2,36.735,49,29,1,28,76,134,180,5.700000e-02,31.6,NP_312540.2,NP_000107.1
94,NP_000112.1,NP_310235.1,39.535,43,25,1,69,111,17,58,4.400000e-01,28.1,NP_310235.1,NP_000112.1
113,NP_000134.2,NP_310344.1,60.606,462,179,2,48,508,2,461,0.000000e+00,567.0,NP_310344.1,NP_000134.2
114,NP_000135.2,NP_312764.1,30.159,63,42,2,130,192,37,97,2.510000e-04,36.6,NP_312764.1,NP_000135.2
